In [1]:
# ruff: noqa: F401
import _config

from continual_learning.continual_metrics import compute_continual_metrics, get_task_matrices
from tools.io_utils import load_json

import os

from typing import Dict, Any

# Results directory
RESULTS_CAPACITIES_DIR = "results-capacities"

General functions to be used in the processing of the results:

In [2]:
def read_experiment_results(experiment_dir: str) -> Dict[str, Any]:
    """
    Read the results of an experiment.
    """
    history_json_path = os.path.join(experiment_dir, "history.json")
    history = load_json(history_json_path)
    return history

### 1. Initialisations

In [9]:
from collections import defaultdict
import numpy as np

TEST_DIR = "cifar10@5-e1"

experiment_dirs = os.listdir(os.path.join(RESULTS_CAPACITIES_DIR, TEST_DIR, "models"))

# Group experiment data
experiment_data = defaultdict(lambda: defaultdict(list))

for experiment_name in experiment_dirs:
    experiment_dir = os.path.join(RESULTS_CAPACITIES_DIR, TEST_DIR, "models", experiment_name)

    # Get the settings from the experiment name
    try:
        tau_at, seed = experiment_name.split("-")
        tau_value = tau_at.lstrip("tau")
        experiment_results = read_experiment_results(experiment_dir)

        # Get the training metrics
        training_metrics = experiment_results["training_metrics"]["validation"]
        accuracies = [task_data["accuracy"] for task_data in training_metrics.values()]
        flatness = [task_data["flatness"] for task_data in training_metrics.values()]

        # Store the metrics in the experiment data
        experiment_data[f"{tau_value}"]["accuracies"].append(accuracies)
        experiment_data[f"{tau_value}"]["flatness"].append(flatness)
    except:
        print(f"Error with experiment {experiment_name}")
        
# Aggregate the experiment data
experiment_data = {
    key: {
        "accuracies": np.mean(value["accuracies"], axis=0),
        "flatness": np.mean(value["flatness"], axis=0),
    }
    for key, value in experiment_data.items()
}

In [10]:
# Get the setting with best mean accuracy
best_setting = max(experiment_data, key=lambda x: np.mean(experiment_data[x]["accuracies"]))
print(best_setting)

0.08
